In [0]:
%python
from pyspark.sql.functions import udf, col, regexp_replace
from pyspark.sql.types import StringType


def mask_phone_number(phone_number):
    #replace ISD from the number 
    phone_number = phone_number.replace('+', '')
    #keep only number and remove all su=ymbols and other
    phone_number = ''.join(filter(str.isdigit, phone_number))
    if phone_number and len(phone_number) >= 10:
        new_phone_number = phone_number[:4] + '*' * (len(phone_number) - 4)
        return new_phone_number
    return phone_number

mask_phone_number_udf = udf(mask_phone_number, StringType())


# Example DataFrame to demonstrate the UDF
df = spark.createDataFrame([('+1234567890897',), ('0987654321',)], ['phone_number'])

df = df.withColumn('masked_phone_number', mask_phone_number_udf(col('phone_number')))

#display(df)

In [0]:
# %sql
# CREATE OR REPLACE FUNCTION dev_edh.dummy.mask_phone_number(phone_number STRING)
#   RETURNS STRING
#   LANGUAGE PYTHON
#   AS $$
#     def mask_phone_number(phone_number):
#         from pyspark.sql.functions import udf
#         from pyspark.sql.types import StringType

#         if len(phone_number) >= 10:
#             return phone_number[:4] + '****'
#             return new_phone_number
#         else:
#             return phone_number
#       $$